In [ ]:
import os
import re

def process_text(text):
    """
    Xử lý nội dung văn bản để phân tách và gán các cột theo định dạng token POS NER (BIO).
    """
    tokens = re.split(r'(\s+|<ENAMEX.*?>|</ENAMEX>)', text)
    tag_stack = []  # Stack lưu trữ các nhãn hiện tại
    result = []

    for token in tokens:
        token = token.strip()
        if not token:  # Bỏ qua token trống
            continue

        if token.startswith('<ENAMEX TYPE='):
            # Bắt đầu thẻ ENAMEX -> Lấy nhãn và đẩy vào stack
            tag = re.search(r'TYPE="([^"]+)"', token).group(1)
            tag_stack.append(tag)
        elif token == '</ENAMEX>':
            # Kết thúc thẻ ENAMEX -> Xóa nhãn khỏi stack
            if tag_stack:
                tag_stack.pop()
        else:
            # Tạo dòng dữ liệu với token và nhãn
            if tag_stack:
                current_tag = f'B-{tag_stack[-1]}' if len(tag_stack) == 1 else f'I-{tag_stack[-1]}'
            else:
                current_tag = 'O'
            result.append([token, current_tag])  # POS tag mặc định là 'O'

    return result

def process_folder(input_dir, output_file):
    """
    Xử lý tất cả file .muc trong thư mục và ghi kết quả vào file .txt.
    """
    with open(output_file, 'w', encoding='utf-8') as output:
        for filename in os.listdir(input_dir):
            if filename.endswith('.muc'):
                input_path = os.path.join(input_dir, filename)
                with open(input_path, 'r', encoding='utf-8') as file:
                    text = file.read()

                # Xử lý văn bản thành định dạng token-level
                processed_data = process_text(text)

                # Ghi từng token vào file output
                for token, ner in processed_data:
                    output.write(f"{token} {ner}\n")
                output.write("\n")  # Dòng trống giữa các file

if __name__ == "__main__":
    # Định nghĩa các thư mục input và file output
    input_output_mapping = {
        "/content/VLSP2021/Dev-Muc": "dev-muc.txt",
        "/content/VLSP2021/Train-Muc": "train.txt",
        "/content/Testing/Test": "tests.txt"
    }

    # Xử lý từng thư mục
    for input_dir, output_file in input_output_mapping.items():
        print(f"Processing {input_dir} -> {output_file}")
        process_folder(input_dir, output_file)

    print("Hoàn thành xử lý tất cả các thư mục!")


Processing /content/VLSP2021/Dev-Muc -> dev-muc.txt
Processing /content/VLSP2021/Train-Muc -> train.txt
Processing /content/Testing/Test -> tests.txt
Hoàn thành xử lý tất cả các thư mục!


In [ ]:
from collections import defaultdict, Counter
from tqdm import tqdm

In [ ]:
def load_data(file_path):
    sentences, labels = [], []
    current_sentence, current_labels = [], []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                token, label = line.strip().split()
                # Bỏ qua nếu nhãn là 'O'
                if label != 'O':
                    current_sentence.append(token)
                    current_labels.append(label)
            else:
                if current_sentence:  # Chỉ thêm câu nếu không rỗng
                    sentences.append(current_sentence)
                    labels.append(current_labels)
                current_sentence, current_labels = [], []

    return sentences, labels

In [ ]:
def train_hmm(sentences, labels):
    """
    Học tham số HMM dựa trên token và NER tag, bỏ qua nhãn "O".

    Args:
        sentences: Danh sách các câu, mỗi câu là một danh sách các token.
        labels: Danh sách các nhãn NER tương ứng với các câu.

    Returns:
        Tuple: Gồm các tham số của mô hình HMM:
            - initial_probs: Xác suất khởi đầu của các trạng thái.
            - transition_probs: Xác suất chuyển đổi giữa các trạng thái.
            - emission_probs: Xác suất phát xạ của các token từ các trạng thái.
            - states: Danh sách tất cả các trạng thái NER (trừ "O").
    """
    initial_probs = Counter()
    transition_probs = defaultdict(Counter)
    emission_probs = defaultdict(Counter)
    total_states = Counter()

    print("Bắt đầu huấn luyện mô hình HMM...")
    for sentence, label_seq in tqdm(zip(sentences, labels), total=len(sentences), desc="Đang huấn luyện"):
        # Bỏ qua nhãn "O" trong quá trình training
        filtered_labels = [label for label in label_seq if label != 'O']
        if not filtered_labels:  # Nếu câu chỉ chứa nhãn "O", bỏ qua câu này
            continue

        initial_probs[filtered_labels[0]] += 1  # Đếm trạng thái khởi đầu (bỏ qua "O")
        for i in range(len(filtered_labels)):
            total_states[filtered_labels[i]] += 1  # Đếm số lần xuất hiện của mỗi trạng thái (bỏ qua "O")
            emission_probs[filtered_labels[i]][sentence[label_seq.index(filtered_labels[i])]] += 1  # Đếm số lần token được phát xạ từ trạng thái (bỏ qua "O")
            if i > 0:
                transition_probs[filtered_labels[i - 1]][filtered_labels[i]] += 1  # Đếm số lần chuyển đổi giữa các trạng thái (bỏ qua "O")

    # Chuẩn hóa xác suất
    initial_probs = {k: v / sum(initial_probs.values()) for k, v in initial_probs.items()}
    transition_probs = {k: {k2: v2 / sum(v.values()) for k2, v2 in v.items()} for k, v in transition_probs.items()}
    emission_probs = {k: {k2: v2 / sum(v.values()) for k2, v2 in v.items()} for k, v in emission_probs.items()}

    print("Huấn luyện xong!")
    return initial_probs, transition_probs, emission_probs, list(total_states.keys())

In [ ]:
def viterbi(sentence, initial_probs, transition_probs, emission_probs, states):
    """
    Thuật toán Viterbi để tìm chuỗi nhãn NER có xác suất cao nhất cho một câu, bỏ qua nhãn "O".

    Args:
        sentence: Danh sách các token của câu cần dự đoán.
        initial_probs: Xác suất khởi đầu của các trạng thái.
        transition_probs: Xác suất chuyển đổi giữa các trạng thái.
        emission_probs: Xác suất phát xạ của các token từ các trạng thái.
        states: Danh sách tất cả các trạng thái NER (trừ "O").

    Returns:
        Danh sách các nhãn NER dự đoán cho câu, có thể bao gồm nhãn "O".
    """
    dp = [{} for _ in range(len(sentence))]  # Khởi tạo bảng DP
    path = [{} for _ in range(len(sentence))]  # Khởi tạo bảng lưu trữ đường đi

    # Bước khởi tạo: Xác suất cho trạng thái đầu tiên
    for state in states:
        dp[0][state] = initial_probs.get(state, 0) * emission_probs[state].get(sentence[0], 1e-6)
        path[0][state] = [state]  # Lưu đường đi

    # Bước quy hoạch động: Tính xác suất cho các trạng thái tiếp theo
    for t in range(1, len(sentence)):
        for curr_state in states:
            max_prob, prev_state = max(
                (dp[t - 1][prev_state] * transition_probs[prev_state].get(curr_state, 1e-6) *
                 emission_probs[curr_state].get(sentence[t], 1e-6), prev_state)  # Tính xác suất chuyển đổi và phát xạ
                for prev_state in states
            )
            dp[t][curr_state] = max_prob  # Lưu xác suất cao nhất
            path[t][curr_state] = path[t - 1][prev_state] + [curr_state]  # Lưu đường đi

    # Tìm đường đi tốt nhất: Trạng thái cuối cùng có xác suất cao nhất
    if dp[-1]:  # Kiểm tra xem dp[-1] có trống không (trường hợp câu chỉ chứa "O")
        final_state = max(dp[-1], key=dp[-1].get)
        predicted_labels = path[-1][final_state]  # Lấy chuỗi nhãn dự đoán
    else:
        predicted_labels = ['O'] * len(sentence)  # Nếu câu chỉ chứa "O", gán tất cả nhãn là "O"

    return predicted_labels

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Khai báo biến để lưu trữ các chỉ số
epoch_history = []
accuracy_history = []
f1_history = []  # hoặc các chỉ số khác bạn muốn theo dõi
precision_history = []
recall_history = []

In [ ]:
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score

def evaluate(dev_sentences, dev_labels, hmm_params):
    """
    Đánh giá hiệu suất của mô hình HMM trên tập development.

    Args:
        dev_sentences: Danh sách các câu trong tập development.
        dev_labels: Danh sách các nhãn NER tương ứng với các câu trong tập development.
        hmm_params: Các tham số của mô hình HMM (initial_probs, transition_probs, emission_probs, states).

    Returns:
        None. In ra kết quả đánh giá (precision, recall, F1-score).
    """
    initial_probs, transition_probs, emission_probs, states = hmm_params
    predictions = []

    print("\nĐang đánh giá mô hình HMM trên tập kiểm thử...")

    # Dự đoán nhãn cho các câu trong tập development
    for sentence in tqdm(dev_sentences, desc="Đang dự đoán"):
        pred = viterbi(sentence, initial_probs, transition_probs, emission_probs, states)
        predictions.append(pred)

    # Nối tất cả các nhãn và dự đoán
    y_true = [label for sublist in dev_labels for label in sublist]
    y_pred = [label for sublist in predictions for label in sublist]

    # Tính toán accuracy
    accuracy = accuracy_score(y_true, y_pred)
    print("\nAccuracy:")
    print(f"{accuracy:.4f}")

    # Tính toán classification report (bao gồm micro, macro, weighted avg)
    print("\nClassification Report:")
    report = classification_report(y_true, y_pred, zero_division=0)
    print(report)

    # Tính toán thủ công micro avg, macro avg, và weighted avg nếu cần thêm chi tiết
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='micro', zero_division=0
    )
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        y_true, y_pred, average='weighted', zero_division=0
    )

    # In thêm thông tin chi tiết nếu cần
    print("\nChi tiết kết quả trung bình:")
    print(f"Micro avg - Precision: {precision_micro:.4f}, Recall: {recall_micro:.4f}, F1-Score: {f1_micro:.4f}")
    print(f"Macro avg - Precision: {precision_macro:.4f}, Recall: {recall_macro:.4f}, F1-Score: {f1_macro:.4f}")
    print(f"Weighted avg - Precision: {precision_weighted:.4f}, Recall: {recall_weighted:.4f}, F1-Score: {f1_weighted:.4f}")


In [ ]:
# Main
train_file = '/content/train.txt'
dev_file = '/content/dev-muc.txt'
test_file = '/content/tests.txt'

In [ ]:
train_sentences, train_labels = load_data(train_file)
dev_sentences, dev_labels = load_data(dev_file)

In [ ]:
hmm_params = train_hmm(train_sentences, train_labels)

Bắt đầu huấn luyện mô hình HMM...


Đang huấn luyện: 100%|██████████| 1044/1044 [00:00<00:00, 4089.08it/s]

Huấn luyện xong!


In [ ]:
import os

def count_files_in_directory(directory_path):
  """
  Đếm số lượng file trong một thư mục cụ thể.

  Args:
    directory_path: Đường dẫn đến thư mục cần đếm file.

  Returns:
    Số lượng file trong thư mục.
  """
  file_count = 0
  for filename in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, filename)):
      file_count += 1
  return file_count

# Đường dẫn đến thư mục bạn muốn đếm file
directory_path = "/content/VLSP2021/Train-Muc"

# Đếm số lượng file trong thư mục
number_of_files = count_files_in_directory(directory_path)

# In ra kết quả
print(f"Số lượng file trong thư mục {directory_path}: {number_of_files}")

Số lượng file trong thư mục /content/VLSP2021/Train-Muc: 1050


In [ ]:
evaluate(dev_sentences, dev_labels, hmm_params)


Đang đánh giá mô hình HMM trên tập kiểm thử...


Đang dự đoán: 100%|██████████| 449/449 [02:02<00:00,  3.67it/s]



Accuracy:
0.2080

Classification Report:
                       precision    recall  f1-score   support

            B-ADDRESS       0.00      0.00      0.00        30
           B-DATETIME       0.59      0.40      0.48      2305
      B-DATETIME-DATE       0.46      0.16      0.24       688
 B-DATETIME-DATERANGE       0.18      0.12      0.14       316
  B-DATETIME-DURATION       0.28      0.25      0.27       902
       B-DATETIME-SET       0.12      0.16      0.14        73
      B-DATETIME-TIME       0.13      0.12      0.12        89
 B-DATETIME-TIMERANGE       0.11      0.06      0.08       159
              B-EMAIL       0.00      0.00      0.00         1
              B-EVENT       0.11      0.07      0.09       516
          B-EVENT-CUL       0.54      0.19      0.29       129
     B-EVENT-GAMESHOW       0.00      0.00      0.00       154
      B-EVENT-NATURAL       0.89      0.28      0.43        57
        B-EVENT-SPORT       0.17      0.34      0.23        97
           B

In [ ]:
# Load dữ liệu test
test_sentences, test_labels = load_data(test_file)

# Đánh giá mô hình trên tập test
evaluate(test_sentences, test_labels, hmm_params)


Đang đánh giá mô hình HMM trên tập kiểm thử...


Đang dự đoán: 100%|██████████| 309/309 [01:36<00:00,  3.19it/s]



Accuracy:
0.2126

Classification Report:
                       precision    recall  f1-score   support

            B-ADDRESS       0.00      0.00      0.00       204
           B-DATETIME       0.41      0.31      0.35      1283
      B-DATETIME-DATE       0.47      0.18      0.26      1087
 B-DATETIME-DATERANGE       0.26      0.28      0.27       327
  B-DATETIME-DURATION       0.26      0.27      0.26       716
       B-DATETIME-SET       0.00      0.00      0.00        13
      B-DATETIME-TIME       0.26      0.15      0.19        73
 B-DATETIME-TIMERANGE       0.19      0.07      0.10       165
              B-EMAIL       0.00      0.00      0.00         2
              B-EVENT       0.26      0.18      0.21       756
          B-EVENT-CUL       0.47      0.24      0.31        34
     B-EVENT-GAMESHOW       0.18      0.08      0.11       248
      B-EVENT-NATURAL       0.80      0.17      0.28        24
        B-EVENT-SPORT       0.50      0.24      0.32       491
            

In [ ]:
# In bảng xác suất chuyển trạng thái
initial_probs, transition_probs, emission_probs, states = hmm_params
for current_state, next_states in transition_probs.items():
    print(f"Xác suất chuyển trạng thái từ {current_state}:")
    for next_state, prob in next_states.items():
        print(f"  - Đến {next_state}: {prob:.4f}")  # In với 4 chữ số thập phân
    print()

Xác suất chuyển trạng thái từ B-LOCATION:
  - Đến B-LOCATION: 0.7236
  - Đến B-DATETIME-DATE: 0.0044
  - Đến B-ORGANIZATION: 0.0596
  - Đến B-PERSON: 0.0639
  - Đến B-MISCELLANEOUS: 0.0079
  - Đến B-DATETIME-DURATION: 0.0018
  - Đến B-DATETIME: 0.0258
  - Đến B-PRODUCT: 0.0105
  - Đến B-QUANTITY: 0.0189
  - Đến B-LOCATION-GPE: 0.0366
  - Đến I-LOCATION-GPE: 0.0002
  - Đến B-QUANTITY-NUM: 0.0103
  - Đến I-QUANTITY-NUM: 0.0020
  - Đến B-ORGANIZATION-SPORTS: 0.0013
  - Đến B-EVENT: 0.0039
  - Đến B-QUANTITY-ORD: 0.0010
  - Đến B-PERSONTYPE: 0.0160
  - Đến B-LOCATION-GEO: 0.0020
  - Đến B-QUANTITY-CUR: 0.0013
  - Đến B-LOCATION-STRUC: 0.0007
  - Đến I-QUANTITY-DIM: 0.0002
  - Đến B-DATETIME-DATERANGE: 0.0004
  - Đến B-QUANTITY-PER: 0.0005
  - Đến B-EVENT-GAMESHOW: 0.0004
  - Đến B-QUANTITY-DIM: 0.0004
  - Đến B-EVENT-SPORT: 0.0005
  - Đến B-DATETIME-TIME: 0.0015
  - Đến B-DATETIME-TIMERANGE: 0.0002
  - Đến B-QUANTITY-AGE: 0.0010
  - Đến I-LOCATION: 0.0004
  - Đến B-ADDRESS: 0.0004
  - Đến 

In [ ]:
new_sentence = "Sofascore là hãng thống kê hàng đầu thế giới, do người Croatia sáng lập năm 2010 và có hàng chục triệu người dùng. Theo mô hình của hãng, Xuân Son nhận điểm 10 và dĩ nhiên là cầu thủ hay nhất trận. Vĩ Hào đứng thứ hai với 8,4 điểm, nhờ một bàn và một đường kiến tạo, còn Nguyễn Quang Hải nhận 8,2 điểm với một bàn."

In [ ]:
# Tiền xử lý câu văn
processed_sentence = [token for token, _ in process_text(new_sentence)]

# Dự đoán nhãn
predicted_labels = viterbi(processed_sentence, *hmm_params)

# Hiển thị kết quả
for token, label in zip(processed_sentence, predicted_labels):
    print(f"{token} {label}")

Sofascore B-PERSONTYPE
là B-PERSONTYPE
hãng B-PERSONTYPE
thống B-PERSONTYPE
kê B-PERSON
hàng B-PERSON
đầu B-DATETIME
thế B-DATETIME
giới, B-MISCELLANEOUS
do B-MISCELLANEOUS
người B-MISCELLANEOUS
Croatia B-LOCATION-GPE
sáng B-DATETIME-TIMERANGE
lập I-DATETIME-DATE
năm B-DATETIME-SET
2010 B-DATETIME-SET
và B-DATETIME-SET
có B-DATETIME-SET
hàng B-QUANTITY-NUM
chục B-DATETIME-DURATION
triệu B-MISCELLANEOUS
người B-MISCELLANEOUS
dùng. B-MISCELLANEOUS
Theo B-MISCELLANEOUS
mô B-MISCELLANEOUS
hình B-MISCELLANEOUS
của B-MISCELLANEOUS
hãng, B-MISCELLANEOUS
Xuân B-LOCATION-GPE
Son B-LOCATION-STRUC
nhận B-LOCATION-STRUC
điểm B-LOCATION-STRUC
10 B-DATETIME-DURATION
và B-DATETIME-SET
dĩ I-DATETIME-DATE
nhiên B-LOCATION-STRUC
là B-LOCATION-STRUC
cầu B-LOCATION-STRUC
thủ I-LOCATION-GPE
hay I-LOCATION-GPE
nhất B-QUANTITY-ORD
trận. B-QUANTITY-ORD
Vĩ B-QUANTITY-ORD
Hào B-QUANTITY-ORD
đứng B-QUANTITY-ORD
thứ B-QUANTITY-ORD
hai B-QUANTITY-NUM
với B-QUANTITY-NUM
8,4 B-QUANTITY-NUM
điểm, B-QUANTITY-NUM
nhờ B